In [ ]:
%tensorflow_version 2.x
# Подключаем необходимые питонячие библиотеки
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [ ]:
# Загружаем обученную модель с github.com. Файл представляет собой архив, в котором содержатся файлы хранящие структуру нейронной сети и ее коэффициенты
# Для загрузки модели сначала скачаем архив сюда, то есть на сервер, где запущен наш ноутбук. Используем команду wget (см. https://www.gnu.org/software/wget/):
!wget https://github.com/ilja2209/lstm_reviews_model/raw/main/lstm_reviews_en.tar.gz

In [ ]:
# Скачали. Теперь нужно архив разархивировать:
!tar -xvf lstm_reviews_en.tar.gz

In [ ]:
# Удаляем архив, ибо он нам уже не нужен и чтобы место на диске нашего сервера не занимал:
!rm -f lstm_reviews_en.tar.gz

In [ ]:
# Также не забываем скачать предобученный токенайзер, кторый содержит все проиндексированные слова из обучающей выборки. Это нужно для того, чтобы было однозначное соответствие
# индексов слов из обучающей выборки и тестовых примеров. Иначе нейронная сеть неправильно отработает
# Для загрузки всё также используем wget:
!wget https://github.com/ilja2209/lstm_reviews_model/raw/main/tokenizer.pickle

Теперь загружаем наши хранящиеся на диске модель и токенайзер в оперативную память, чтобы с ними дальше можно было работать:

In [ ]:
# Загружаем модель:
model = keras.models.load_model("lstm_reviews_en")

In [ ]:
# Загружаем tokenizer:
with open('tokenizer.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

Вот теперь мы всё приготовили для дальнейшей работы. У нас есть загруженные в память токенайзер и модель. Теперь напишем парочку функций, делающих наш код чище и после этого можем эксперементировать.  

In [ ]:
# Напишем функцию, на вход которой мы подаем текст, а на выходе получаем вектор, который потом подадим на вход нейронной сети для инференса:
def text2Sequence(text):
  max_review_len = 100 #Количество слов (чисел, нейронов входного слоя), подаваемых на вход нейронной сети
  sequence = tokenizer.texts_to_sequences([text])
  return pad_sequences(sequence, maxlen=max_review_len)


In [ ]:
# Напишем функцию, которая выход нейронной сети преобразует в человеко-читаемый ответ:
def result2Text(result):
  if result[[0]] < 0.5:
    return 'Отзыв отрицательный'
  else:
    return 'Отзыв положительный'

Ну вот и всё! С этого места можем эксперементировать с ниференсом. Всё, что описано выше достаточно выполнить один раз, дальше можно просто писать код, который будет подавать на вход модели текст комментария и получать на выходе его тон. Например:

In [ ]:
# Преобразуем текст в числовое представление
seq = text2Sequence('Beautiful young woman turned down a proposal')
# Выполняем подаем на вход модели вектор, полученный ранее и сразу же преобразуем результат работы нейронной сети в текстовый результат
result = result2Text(model.predict(seq))
# Выводим результат на экран
print(result)

А вот теперь можно попробовать взять список комментариев и, например, посчитать количество положительных и отрицатеельных отзывов. Всё!